In [1]:
import pandas as pd
import datetime
import os
import numpy as np

In [2]:
os.getcwd()

'/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post'

In [3]:
store_list=pd.read_excel('/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/SOTF Wave 1 Store IDs_8.20.xlsx',
                         dtype=str,sheetname="Stores_6_8")
store_set_wave_1=set(store_list['SOTF'].unique().tolist())

In [4]:
store_wide_sales=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="sales")

In [5]:
store_wide_trans=pd.read_excel("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/wide_sales_date2018-09-01.xlsx",
                               dtype=str,sheetname="transactions")

In [6]:
First_week_2018=datetime.date(2018,5,19)
First_week_2017=datetime.date(2017,5,20)

weeks_2018=[str(First_week_2018+datetime.timedelta(days=i*7)) for i in range(8)]
weeks_2017=[str(First_week_2017+datetime.timedelta(days=i*7)) for i in range(8)]

In [7]:
SOTF_sales=store_wide_sales[['location_id']+weeks_2017+weeks_2018]
SOTF_sales['Wave_1']=SOTF_sales['location_id'].apply(lambda x: x in store_set_wave_1)

SOTF_trans=store_wide_trans[['location_id']+weeks_2017+weeks_2018]
SOTF_sales['Wave_1']=SOTF_sales['location_id'].apply(lambda x: x in store_set_wave_1)

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [8]:
for col in SOTF_trans.columns.tolist()[1:]:
    SOTF_trans[col]=SOTF_trans[col].astype(int)
    
for col in SOTF_sales.columns.tolist()[1:]:
    SOTF_sales[col]=SOTF_sales[col].apply(lambda x: np.round(float(x),2))

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:5: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """


In [9]:
SOTF_sales['inclusion']=np.nan

for i in range(len(SOTF_sales)):
    ind=1
    for j in range(1,17):
        if SOTF_sales.iloc[i,j]==0:
            ind=0
    SOTF_sales['inclusion'][i]=ind

/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  """Entry point for launching an IPython kernel.
/usr/local/lib/python3.6/site-packages/ipykernel_launcher.py:8: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  
/usr/local/lib/python3.6/site-packages/IPython/core/interactiveshell.py:2847: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  exec(code_obj, self.user_global_ns, self

In [10]:
SOTF_sales.shape

(1513, 19)

# Rewards

In [11]:
start_2017=datetime.datetime.strptime(weeks_2017[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
start_2018=datetime.datetime.strptime(weeks_2018[0],"%Y-%m-%d").date()-datetime.timedelta(days=6)
end_2017=datetime.datetime.strptime(weeks_2017[7],"%Y-%m-%d").date()
end_2018=datetime.datetime.strptime(weeks_2018[7],"%Y-%m-%d").date()

In [12]:
data_from_SP=pd.read_csv("/home/jian/Projects/Big_Lots/Loyal_members/loyalty_sales_data/From_Sp/combinedtransactions_0811.csv",
                        dtype=str,usecols=['customer_id_hashed','transaction_date','transaction_id','location_id','total_transaction_amt'])
data_from_SP['transaction_date']=data_from_SP['transaction_date'].apply(lambda x: datetime.datetime.strptime(x,"%Y-%m-%d").date())
data_from_SP=data_from_SP[(data_from_SP['transaction_date']>=start_2017) & (data_from_SP['transaction_date']<=end_2017) |\
                         (data_from_SP['transaction_date']>=start_2018) & (data_from_SP['transaction_date']<=end_2018)]
data_from_SP=data_from_SP.drop_duplicates()

In [13]:
def week_end_func(x):
    if x.weekday()==6:
        week=x+datetime.timedelta(days=6)
    else:
        diff=x.weekday()
        week=x+datetime.timedelta(days=5-diff)
    return week


In [14]:
data_from_SP['week_end_date']=data_from_SP['transaction_date'].apply(lambda x: week_end_func(x))
# data_from_SP=data_from_SP[data_from_SP['location_id'].isin(inclusions_stores)]
data_from_SP['total_transaction_amt']=data_from_SP['total_transaction_amt'].astype(float)

In [15]:
rewards_by_week_sales=data_from_SP.groupby(['location_id','week_end_date'])['total_transaction_amt'].sum().to_frame().reset_index()
rewards_by_week_trans=data_from_SP.groupby(['location_id','week_end_date'])['total_transaction_amt'].count().to_frame().reset_index()


In [26]:
len(data_from_SP['transaction_date'].unique())

112

In [29]:
data_from_SP['transaction_date'].max()

datetime.date(2018, 7, 7)

In [30]:
data_from_SP['transaction_date'].min()

datetime.date(2017, 5, 14)

# Combine

In [17]:
output=SOTF_sales[['location_id']]
output=output.rename(columns={"SOTF":"location_id"})
SOTF_sales.head(2)

,location_id,2017-05-20,2017-05-27,2017-06-03,2017-06-10,2017-06-17,2017-06-24,2017-07-01,2017-07-08,2018-05-19,2018-05-26,2018-06-02,2018-06-09,2018-06-16,2018-06-23,2018-06-30,2018-07-07,Wave_1,inclusion
0,0,0.0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.0,0.00,0.00,0.00,0.00,0.0,0.0,0.00,0.0,0.0
1,1,74072.4,88270.91,91990.34,77190.27,76737.68,70746.61,84674.98,98495.28,72280.5,92486.88,106743.71,88619.43,87176.25,82495.4,82604.5,102354.41,1.0,1.0


In [18]:
output_store_total_sales=pd.merge(output,SOTF_sales,on="location_id",how="left")
output_store_total_trans=pd.merge(output,SOTF_trans,on="location_id",how="left")

In [19]:
output_store_total_trans.shape

(1513, 17)

In [20]:
rewards_by_week_trans=rewards_by_week_trans.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()
rewards_by_week_sales=rewards_by_week_sales.pivot(index="location_id",columns="week_end_date",values="total_transaction_amt").reset_index()


In [21]:
output_store_rewards_sales=pd.merge(output,rewards_by_week_sales,on="location_id",how="left")
output_store_rewards_trans=pd.merge(output,rewards_by_week_trans,on="location_id",how="left")

In [22]:
output_store_rewards_sales=output_store_rewards_sales.fillna(0)
output_store_rewards_trans=output_store_rewards_trans.fillna(0)

In [23]:
writer=pd.ExcelWriter("/home/jian/Projects/Big_Lots/Analysis/2018_Q2_Post/SOTF_Post/"+"BL_35_stores_opened_20180608_Inclusions_JL_"+str(datetime.datetime.now().date())+".xlsx",engine="xlsxwriter")


In [24]:
output_store_total_sales.to_excel(writer,"store_total_sales",index=False)
output_store_total_trans.to_excel(writer,"store_total_trans",index=False)
output_store_rewards_sales.to_excel(writer,"store_rewards_sales",index=False)
output_store_rewards_trans.to_excel(writer,"store_rewards_trans",index=False)
writer.save()